# Principal Stresses and Axes

This notebook computes the principal stresses and principal directions (rotation matrix) of a given symmetric stress tensor.

**How to use:**
1. Run the cell below to define the function.
2. Modify `sigma` as needed and run the execution cells to see the results.


In [19]:
import numpy as np
np.set_printoptions(precision=3, suppress=True)

In [20]:
# Input: your tensor (MPa) - you are welcome to modify but keep symmetrical
sigma = np.array([[350.000,   86.603,  173.205],
                  [ 86.603,  450.000, -100.000],
                  [173.205, -100.000,  300.000]])

print("User-defined stress tensor (MPa):\n", sigma)

User-defined stress tensor (MPa):
 [[ 350.      86.603  173.205]
 [  86.603  450.    -100.   ]
 [ 173.205 -100.     300.   ]]


In [21]:
# Perform eigen-decomposition
w, V = np.linalg.eigh(sigma)   # w eigenvalues in ascending order, V has eigenvectors as columns

# Sort the eigenvalues and eigenvectors so σ₁ ≥ σ₂ ≥ σ₃
idx = np.argsort(w)[::-1]      # indices that result in descending eigenvalues
w = w[idx]                     # reorder eigenvalues
V = V[:, idx]                  # reorder eigenvectors accordingly

print("\nPrincipal stresses (MPa):", w)


Principal stresses (MPa): [500. 500. 100.]


In [22]:
# Rotation matrix: transposition do that rows of Q [eigenvectors] are principal directions in the original basis
Q = V.T

# Ensure that the rotation matrix is right-handed (i.e. the coordinate system purely rotates without deformation)
if np.linalg.det(Q) < 0:
    Q[-1, :] *= -1

print("\nRotation matrix Q:\n", Q)


Rotation matrix Q:
 [[ 0.607  0.783  0.134]
 [ 0.506 -0.512  0.694]
 [ 0.612 -0.354 -0.707]]


In [24]:
# Stress tensor in the new base by multiplication with the rotation matrix
sig_p_diag = Q @ sigma @ Q.T

print("\nDiagonalized stress tensor (MPa):\n", sig_p_diag)


Diagonalized stress tensor (MPa):
 [[500.   0.   0.]
 [ -0. 500.  -0.]
 [  0.  -0. 100.]]
